In [1]:
## Imports. This contains some imports not required for this notebook. 

import sys
import os
import numpy as np
from matplotlib import pyplot as plt
from matplotlib import pylab                   # Needed to plot histograms.
from astropy.io import fits                    # Need this if you want to use astropy.io io objects.
from astropy.stats import mad_std              # The median absolute deviation, a more robust estimator than std.
from PIL import Image
from scipy.ndimage import gaussian_filter
from scipy import signal

'''Filter out warnings. May or may not need this. But shouldn't hurt to put it in.
This is the "new" code for avoiding warnings-- seems to work better. Copied from an SDSS notebook.'''
import warnings
warnings.filterwarnings('ignore')

sys.path.append(r'C:\Users\owenm\Documents\ASTR 21200 2019\Software\a212pylibs\datapype')

from datafits import DataFits

In [2]:
# Specify the path to the data directory.
datapath = r'C:\Users\owenm\Documents\ASTR 21200 2019\Data\Project\Last'
savepath = r'C:\Users\owenm\Documents\ASTR 21200 2019\Data\BadPixelsFitss'

In [ ]:
## List files in the the data directory.

whichpath = datapath

## For ALL the files in the directory.
# allfiles = [f for f in os.listdir(datapath)]

exposures = ['bias','020s', '040s', '100s', '180s']
allfiles = [0] * 5

## Various list comprehensions can pick out files with particular characteristics.
for i,exposure in enumerate(exposures):
    allfiles[i] = [f for f in os.listdir(whichpath) if '.fit' in f and (exposure in f) and 'mdark' not in f]

for i in range(len(exposures)):    
    allfiles[i] = sorted(allfiles[i])       ## This is necessary on my Mac, may not be for others?
    for j in range(len(allfiles[i])):
        print(j, allfiles[i][j])
    

In [ ]:
## Construct a list of files you wish to view and/or process.

acceptlist = True             # True if you want to accept all the files in allfiles.
contiguous = True             # True if you want to accept a contiguous subset of allfiles.
startfile, endfile = 0,9      # The first and last files in a contiquous subset.
flist = [0,3,4,5,6,7,8]        # An explicit list of the files you want to accept.
files = [0] * 5

if acceptlist == True:
    for i in range(len(files)):
        files[i] = allfiles[i]
        for j in range(len(files[i])):
            print(j, files[i][j])
else:
    if contiguous == True:
        for i in range(len(files)):
            files[i] = allfiles[i][startfile:endfile+1]
            for j in range(len(files[i])):
                print(j, files[i][j])
    else:
        #files = []
        for i in range(len(files)):
            for j in range(len(flist)):
                files[i].append(allfiles[flist[j]])
            for j in range(len(files[i])):
                print(j, flist[j], files[i][j])

# Task 1. Creating Faulty Non-Noisy Pixels Fits

In [ ]:
'''Make a stack of images, a list of headers from those images, and calculate some medians and stds
that will help set autoscaling parameters.'''


image = [0] * 5
imedian = [0] * 5
istd = [0] * 5
madstd = [0] * 5
headlist = [''] * 5
for i in range(5):
    image[i] = np.zeros((len(files[i]), 1024,1024))
    imedian[i] = np.zeros((len(files[i])))          
    istd[i] = np.zeros((len(files[i])))              
    madstd[i] = np.zeros((len(files[i]))) 
    headlist[i] = []  
    print('image.shape =', image[i].shape)
    for j in range(len(files[i])):


#############################################################################
    # Use this code block if you want to use the standard astropy.io package.
        fitsfilename = os.path.join(datapath, files[i][j])    # Full path to a fitsfile.
        hdulist = fits.open(fitsfilename)                  # Open a fits file as an hdulist object.
        hdu0 = hdulist[0]                                  # Define a fits object as the 0th hdu in the fitsfile.
        image[i][j] = hdu0.data * 1.0                         # Define image in stack as float of data in the 0th hdu.
        headlist.append(hdu0.header)                       # Append the header of the fits object to the header list.
        print('')
        print(j,files[i][j])
###############################################################################    
    
    ## Calculate some medians and stds to use in autoscaling that aren't unduly biased by extreme values.
    ## Optionally, print out masked and unmasked values to explore the effects of extreme values on the statistics.
    
        madstd[i][j] = mad_std(image[i][j],ignore_nan=True)
        imedian[i][j] = np.nanmedian(image[i][j])
        istd[i][j] = np.nanstd(image[i][j])

In [ ]:
#Creating a map of how often the pixels have been bad

bads = [0] * 5
imedian2 = [0] * 5
istd2 = [0] * 5
b = [0] * 5
pix = [0] * 5
ctr = [0] * 5
for i in range(5):
    bads[i] = np.zeros((1024,1024))

    imedian2[i] = np.zeros((len(files[i])))           # 1D numpy array to hold array medians.
    istd2[i] = np.zeros((len(files[i])))              # 1D numpy array to hold array stds.

    for j in range(len(files[i])):
        imedian2[i][j] = np.nanmedian(image[i][j,200:,:])
        istd2[i][j] = np.nanstd(image[i][j,200:,:])
        #sets the median and std arrays to hold the median and std of each image

    for j in range(len(image[i])):
        istand = image[i][j] - imedian2[i][j]
        bad1 = (istand > istd2[i][j]) * 1
        #check if 1 standard deviation above the median
        bad2 = (istand < -istd2[i][j]) * 1
        #check if 1 standard deviation below the median
        bad = bad1 + bad2
        #will be 1 if one standard deviation off (above or below), 0 otherwise
        bads[i] += bad
    #counts the number

    #print(bads)
    #print(bads[:,1:])
    b[i] = bads[i][:, 200:]
    print(b[i])
    pix[i] = np.zeros((1024,824))
    #ctr = 0
    for j in range(1024):
        for k in range(824):
            if b[i][j,k] >= len(files[i]) * 0.6:
                ctr[i] += 1
                pix[i][j,k] = 1
                #print(j,k)
    #checks number of images for which a pixel is in "bads"
    #if  >20 times bad, sets the pixel's corresponding point in "pix" to 1

    print(str(ctr[i]/(1024 * 824)))
#gives ratio of flaged pixels to all pixels

The above array gives the number of times, for each pixel, it has been deemed "bad" in the list of 300 darks.

If we check pixels that have been bad at least once: 49.2% of pixels meet this criteria If we check pixels that have been bad at least twice: 18.5% of pixels meet this criteria If we check pixels that have been bad at least 3 times: 8.28% of pixels meet this criteria If we check pixels that have been bad at least 4 times: 5.21% of pixels meet this criteria ...

Following code makes list

In [ ]:
pix3 = [0] * 5
mask3 = [0] * 5
for i in range(5):
    pix3[i] = np.zeros((1024,824))
    #makes 2D a zero array that will become our final list of pixels to smooth over
    mask3[i] = np.zeros((len(files[i]),1024,824))
    #zero array that will become a stack of masks
    for j in range(1024):
        for k in range(824):
            if pix[i][j,k] == 1:
            #checks if pixel has been flagged by first algorithm, ignore if not
            #this next series of if statements considers several different possible pixel locations, with the goal of setting a "sur"
            #"sur" is a 2D array that is basically the pixel values of the surronding area
                if j >= 5 and j < 1019:
                    #all of these use a row value between 5 and 1019
                    if k >= 5 and k < 819:
                        sur = image[i][:,j-5:j+5,200+k-5:200+k+5]
                        #these pixels are in places where we can easily check the surronding area
                    elif k < 5:
                        sur = np.append(image[i][:,j-5:j+5,200:200+k], image[i][:,j-5:j+5,-10+k:])
                        #if in a colum <5, wraps around the side to fill in the rest of the neighborhood
                    else:
                        sur = np.append(image[i][:,j-5:j+5,k:], image[i][:,j-5:j+5,200:210-1024+k])
                        #column in last 5, wraps around to fill in rest of neighborhood
                elif j < 5:
                    #all of these are for a row value less than 5
                    if k >= 5 and k < 819:
                        sur = np.append(image[i][:,:j,200+k-5:200+k+5], image[i][:,-10+j:,200+k-5:200+k+5])
                        #accaptable column value, only need to wrap around top-bottom (rows)
                    elif k < 5:
                        sur = np.append(image[i][:,:j,200:200+k], image[i][:,-10+j:,-10+k:])
                        #in a corner, wraps around to fill in both dimensions
                    else:
                        sur = np.append(image[i][:,:j,k:], image[i][:,-10+j:,200:210-1024+k])
                        #second corner
                else:
                    #row values in the last 5
                    if k >= 5 and k < 819:
                        sur = np.append(image[i][:,j:,200+k-5:200+k+5], image[i][:, :10-1024+j,200+k-5:200+k+5])
                        #accaptable column value, only need to wrap around top-bottom (rows)
                    elif k < 5:
                        sur = np.append(image[i][:,j:,200:200+k], image[i][:, :10-1024+j,-10+k:])
                        #third corner, wrap around in both dimensions
                    else:
                        sur = np.append(image[i][:,j:,k:], image[i][:, :10-1024+j,200:210-1024+k])
                        #fourth corner
                surmed = np.nanmedian(sur)
                #sets surmed to median value of the surronding area
                surstd = np.nanstd(sur)
                #sets surstd to standard deviation of surronding area
                pmed = np.nanmedian(image[i][:,j,200+k])
                #finds the median value of the pixel accross all 32 images
                pix3[i][j,k] = abs(surmed - pmed) > surstd




In [ ]:
#Stack the arrays of noisy pixels

fbpix = np.zeros((5,1024,1024))

pixhelp2 = np.zeros((1024,200))
for i in range(5):
    fbpix[i,:,:] = np.append(pixhelp2,pix3[i], axis=1)


dims = [5,1024,1024]
dpixm = np.zeros(dims)
dpixm[0,:,:]= fbpix[0,:,:]
dpixm[1,:,:]= fbpix[1,:,:]
dpixm[2,:,:]= fbpix[2,:,:]
dpixm[3,:,:]= fbpix[3,:,:]
dpixm[4,:,:]= fbpix[4,:,:]

In [ ]:
#creating a workable data format for our boolean 2D Array
hdumy = fits.PrimaryHDU(dpixm)

#Turning Workable Object into a fits
outd = fits.HDUList([hdumy])

## Construct a name and 'save directory' for the dark-subtracted, flatfielded image.
outname = 'dead_pixels' + 'test1 ' + '.fits'
outpath = savepath
print('outpath =',outpath)
outf = os.path.join(outpath,outname)
print('outname =',outname)

In [ ]:
## Now save the file (or not).
print(outf)
yes_or_no = input('Save file? Enter "y" or "n":')
if yes_or_no == 'y':
    outd.writeto(outf, overwrite = False)
    print( outname + ' has been saved.')
else:
    print( 'OK-- file was not saved.')

# outd.save(outf)

## Making masks

In [ ]:
# Make a median dark image from the image stack
dark = np.nanmedian(image, axis=0)
# make a bias file
bias = dark.copy()

# take a median row slice and smooth it
rowslice = np.median(bias, axis=0)
# smooth_rowslice = signal.medfilt(rowslice, kernel_size=5)
# smooth_rowslice = signal.savgol_filter(smooth_rowslice, window_length=101, polyorder=1)

In [ ]:
#making initial mask
for i in range(1024):
    for j in range(824):          
            # sets to 1 if the median pixel value is more than one standard deviation away from the median of the surronding area
            if pix3[i,j] == 1:
                mask3[:,i,j] = image[:,i,200+j] - rowslice[200+j]
                #if pix3 is 1, sets the mask to be the value of the original image minus that value of the neighborhood median

In [ ]:
#masking for pix3 

maskhelp2 = np.zeros((len(files),1024,200))
mask6 = np.append(maskhelp2,mask3, axis=2)

smoothed3 = image-mask6

## creating and saving dead pixels fits

In [ ]:
#creating a workable data format for our boolean 2D Array
hdumy = fits.PrimaryHDU(pix3)

#Turning Workable Object into a fits
outd = fits.HDUList([hdumy])

## Construct a name and 'save directory' for the dark-subtracted, flatfielded image.
outname = 'bad_pixels' + '(INSERT NUMBER OR OTHER THING!!!!! OR ITLL BREAK SOME STUFF) 5' + '.fits'
outpath = savepath
print('outpath =',outpath)
outf = os.path.join(outpath,outname)
print('outname =',outname)

In [ ]:
## Now save the file (or not).
print(outf)
yes_or_no = input('Save file? Enter "y" or "n":')
if yes_or_no == 'y':
    outd.writeto(outf, overwrite = False)
    print( outname + ' has been saved.')
else:
    print( 'OK-- file was not saved.')

# outd.save(outf)

# Task 2 Creating Fits of Noisy Pixels




In [ ]:
## List files in the the data directory.

whichpath = datapath

## For ALL the files in the directory.
# allfiles = [f for f in os.listdir(datapath)]

## Various list comprehensions can pick out files with particular characteristics.
allfiles = [f for f in os.listdir(whichpath) if '.fit' in f and ('bias.' in f) and 'mdark' not in f]

allfiles = sorted(allfiles)       ## This is necessary on my Mac, may not be for others?
for i in range(len(allfiles)):
    print( i, allfiles[i])
    

In [ ]:
## Construct a list of files you wish to view and/or process.

acceptlist = True             # True if you want to accept all the files in allfiles.
contiguous = True             # True if you want to accept a contiguous subset of allfiles.
startfile, endfile = 0,9      # The first and last files in a contiquous subset.
flist = [0,3,4,5,6,7,8]        # An explicit list of the files you want to accept.

if acceptlist == True:
    files = allfiles
    for i in range(len(files)):
        print(i, files[i])
else:
    if contiguous == True:
        files = allfiles[startfile:endfile+1]
        for i in range(len(files)):
            print( i, files[i])
    else:
        files = []
        for i in range(len(flist)):
            files.append(allfiles[flist[i]])
        for i in range(len(files)):
            print( i, flist[i], files[i])

In [ ]:
#In dah_functions2 but my imports are being weird so i did it.
def stackfits(whichpath,files):
    '''
    Make a stack of images, a list of headers from those images, and 1D arrays for medians and mad_stds.
    whichpath:   The path to the fits files.
    files:  A list of files.
    '''

    ## Load the header of the first file in the list "files".
    df = DataFits()                                                   # Create a DataPype DataFits io object.
    df.loadhead(os.path.join(whichpath,files[0]))                # Loads just the header of the first file.
    ## Make variables for the numbers of rows and columns.
    rows, cols = df.header['naxis2'], df.header['naxis1']
    print('rows =',rows,'   cols =',cols)

    image = np.zeros((len(files), rows,cols))  # 3D numpy array to hold the stack of images.
    imedian = np.zeros((len(files)))           # 1D numpy array to hold array medians.
    imad = np.zeros((len(files)))            # 1D numpy array to hold Median absolute deviations.

    headlist = []                              # Empty list to hold the headers.
    print('image.shape =', image.shape)

    for i in range(len(files)):

    # ########################################################################   
        # Use this code block if you want to work with DataFits objects
        df = DataFits() 
        df.load(os.path.join(whichpath,files[i]))
        image[i] = df.imageget() * 1.0            # Load image data into numpy arrays and convert to float.
        headlist.append(df.header)
        imedian[i] = np.nanmedian(image[i])
        imad[i] = mad_std(image[i],ignore_nan=True)
        print('')
        print(i, files[i])
    # #########################################################################

    #############################################################################
    #     # Use this code block if you want to use the standard astropy.io package.
    #     fitsfilename = os.path.join(datapath, files[i])    # Full path to a fitsfile.
    #     hdulist = fits.open(fitsfilename)                  # Open a fits file as an hdulist object.
    #     hdu0 = hdulist[0]                                  # Define a fits object as the 0th hdu in the fitsfile.
    #     image[i] = hdu0.data * 1.0                         # Define image in stack as float of data in the 0th hdu.
    #     headlist.append(hdu0.header)                       # Append the header of the fits object to the header list.
    #     print('')
    #     print(i,files[i])
    ###############################################################################   
    return image, headlist, rows, cols, imedian, imad

In [ ]:
nbimage, nbheadlist, nbrows, nbcols, nbimedian, nbimad = stackfits(datapath,allfiles)

In [ ]:
#This is very important

bstackarray = nbimage

In [ ]:
#Analyze the variation of each pixel in the stack

bstackarray_std = np.std(bstackarray, axis=0)

bimg_global_std = np.median(bstackarray_std)

bnoisy_pixels = bstackarray_std > bimg_global_std*1.8

#we identified the pixels across the bias exposures whose values deviate from their respective means by 1.8 standard deviations
# this threshold was chosen based on visual comparisons with 1.7 standev and 1.9 standev

In [ ]:
#Identify and list the pixels that exceed the noise threshold

## plt.imshow(noisy_pixels)

blocs = np.array(np.where(bnoisy_pixels == True))
nbpix_bias = np.zeros((1019,1019)) 
for i in range(1019): #1024
    for j in range(1019): #1024
        if bnoisy_pixels[i,j] == True and j > 5:
            nbpix_bias[i,j] = 1
            

# for i in range(locs.shape[1]):
#     print(locs[0,i],locs[1,i])
    
#this prints a list of noisy pixels

In [ ]:
#Now we repeat the process for 020s
##This can be applied to any subset of raw fits images.
##In this example we used a single data set subdivided into exposure types

somefiles = [f for f in os.listdir(whichpath) if '.fit' in f and '_020s' in f and'stack' not in f]

somefiles = sorted(somefiles)       ## This is necessary on my Mac, may not be for others?


acceptlist = True             # True if you want to accept all the files in somefiles.
contiguous = True             # True if you want to accept a contiguous subset of somefiles.
startfile, endfile = 5,9      # The first and last files in a contiquous subset.
flist = [0,3,4,5,6,7,8]        # An explicit list of the files you want to accept.

n20image, n20headlist, n20rows, n20cols, n20imedian, n20imad = stackfits(datapath,somefiles)


n20stackarray = n20image

n20stackarray_std = np.std(n20stackarray, axis=0)

n20img_global_std = np.median(n20stackarray_std)

n20noisy_pixels = n20stackarray_std > n20img_global_std*1.8


n20locs = np.array(np.where(n20noisy_pixels == True))
npix_020s = np.zeros((1024,1024)) 
for i in range(1024): 
    for j in range(1024): 
        if n20noisy_pixels[i,j] == True and j > 5:
            npix_020s[i,j] = 1

In [ ]:
#Now we repeat the process for 040s
##This can be applied to any subset of raw fits images.
##In this example we used a single data set subdivided into exposure types

somefiles = [f for f in os.listdir(whichpath) if '.fit' in f and '_040s' in f and'stack' not in f]

somefiles = sorted(somefiles)       ## This is necessary on my Mac, may not be for others?


acceptlist = True             # True if you want to accept all the files in somefiles.
contiguous = True             # True if you want to accept a contiguous subset of somefiles.
startfile, endfile = 5,9      # The first and last files in a contiquous subset.
flist = [0,3,4,5,6,7,8]        # An explicit list of the files you want to accept.

n40image, n40headlist, n40rows, n40cols, n40imedian, n40imad = stackfits(datapath,somefiles)


n40stackarray = n40image

n40stackarray_std = np.std(n40stackarray, axis=0)

n40img_global_std = np.median(n40stackarray_std)

n40noisy_pixels = n40stackarray_std > n40img_global_std*1.8


n40locs = np.array(np.where(n40noisy_pixels == True))
npix_040s = np.zeros((1024,1024)) 
for i in range(1024):
    for j in range(1024):
        if n40noisy_pixels[i,j] == True and j > 5:
            npix_040s[i,j] = 1


In [ ]:
#Now we repeat the process for 100s
##This can be applied to any subset of raw fits images.
##In this example we used a single data set subdivided into exposure types

somefiles = [f for f in os.listdir(whichpath) if '.fit' in f and '_100s' in f and'stack' not in f]

somefiles = sorted(somefiles)       ## This is necessary on my Mac, may not be for others?


acceptlist = True             # True if you want to accept all the files in somefiles.
contiguous = True             # True if you want to accept a contiguous subset of somefiles.
startfile, endfile = 5,9      # The first and last files in a contiquous subset.
flist = [0,3,4,5,6,7,8]        # An explicit list of the files you want to accept.

n100image, n100headlist, n100rows, n100cols, n100imedian, n100imad = stackfits(datapath,somefiles)


n100stackarray = n100image

n100stackarray_std = np.std(n100stackarray, axis=0)

n100img_global_std = np.median(n100stackarray_std)

n100noisy_pixels = n100stackarray_std > n100img_global_std*1.8


n100locs = np.array(np.where(n100noisy_pixels == True))
npix_100s = np.zeros((1019,1019)) 
for i in range(1024):
    for j in range(1024): 
        if n100noisy_pixels[i,j] == True and j > 5:
            npix_100s[i,j] = 1

In [ ]:
#Now we repeat the process for 0180s
##This can be applied to any subset of raw fits images.
##In this example we used a single data set subdivided into exposure types

somefiles = [f for f in os.listdir(whichpath) if '.fit' in f and '_180s' in f and'stack' not in f]

somefiles = sorted(somefiles)       ## This is necessary on my Mac, may not be for others?


acceptlist = True             # True if you want to accept all the files in somefiles.
contiguous = True             # True if you want to accept a contiguous subset of somefiles.
startfile, endfile = 5,9      # The first and last files in a contiquous subset.
flist = [0,3,4,5,6,7,8]        # An explicit list of the files you want to accept.

n180image, n180headlist, n180rows, n180cols, n180imedian, n180imad = stackfits(datapath,somefiles)


n180stackarray = n180image

n180stackarray_std = np.std(n180stackarray, axis=0)

n180img_global_std = np.median(n180stackarray_std)

n180noisy_pixels = n180stackarray_std > n180img_global_std*1.8


n180locs = np.array(np.where(n180noisy_pixels == True))
npix_180s = np.zeros((1024,1024)) 
for i in range(1024): 
    for j in range(1024): 
        if n180noisy_pixels[i,j] == True and j > 5:
            npix_180s[i,j] = 1

In [ ]:
#Stack the arrays of noisy pixels

dims = list(nbpix_bias.shape)
dims.append(5)
dims = [5,nbpix_bias.shape[0],nbpix_bias.shape[1]]
npixm = np.zeros(dims)
npixm[0,:,:]= nbpix_bias
npixm[1,:,:]= npix_020s
npixm[2,:,:]= npix_040s
npixm[3,:,:]= npix_100s
npixm[4,:,:]= npix_180s

In [ ]:
#creating a workable data format for our boolean 2D Array
hdumy = fits.PrimaryHDU(npixm)

#Turning Workable Object into a fits
outd = fits.HDUList([hdumy])

## Construct a name and 'save directory' for the dark-subtracted, flatfielded image.
outname = 'noisy_pixels' + '(INSERT NUMBER OR OTHER THING!!!!! OR ITLL BREAK SOME STUFF) 1' + '.fits'
outpath = savepath
print('outpath =',outpath)
outf = os.path.join(outpath,outname)
print('outname =',outname)

In [ ]:
## Now save the file (or not).
print(outf)
yes_or_no = input('Save file? Enter "y" or "n":')
if yes_or_no == 'y':
    outd.writeto(outf, overwrite = False)
    print( outname + ' has been saved.')
else:
    print( 'OK-- file was not saved.')

# outd.save(outf)

# Task 3 Combining all of the noise and bad files list and adding shifting them over.

In [ ]:
datapath2 = r'C:\Users\owenm\Documents\ASTR 21200 2019\Data\BadPixelsFitss'
savepath2 = r'C:\Users\owenm\Documents\ASTR 21200 2019\Data\master'

In [ ]:
## List files in the the data directory.

whichpath = datapath2

## For ALL the files in the directory.
# allfiles = [f for f in os.listdir(datapath)]

## Various list comprehensions can pick out files with particular characteristics.
allfiles = [f for f in os.listdir(whichpath) if 'master' in f]

allfiles = sorted(allfiles)       ## This is necessary on my Mac, may not be for others?
for i in range(len(allfiles)):
    print( i, allfiles[i])
    

In [ ]:
## Construct a list of files you wish to view and/or process.

acceptlist = True             # True if you want to accept all the files in allfiles.
contiguous = True             # True if you want to accept a contiguous subset of allfiles.
startfile, endfile = 0,9      # The first and last files in a contiquous subset.
flist = [0,3,4,5,6,7,8]        # An explicit list of the files you want to accept.

if acceptlist == True:
    files = allfiles
    for i in range(len(files)):
        print(i, files[i])
else:
    if contiguous == True:
        files = allfiles[startfile:endfile+1]
        for i in range(len(files)):
            print( i, files[i])
    else:
        files = []
        for i in range(len(flist)):
            files.append(allfiles[flist[i]])
        for i in range(len(files)):
            print( i, flist[i], files[i])

In [ ]:
fbiasimage = np.zeros((len(files),1024,1024))
f20image = np.zeros((len(files),1024,1024))
f40image = np.zeros((len(files),1024,1024))
f100image = np.zeros((len(files),1024,1024))
f180image = np.zeros((len(files),1024,1024))




for i in range(len(files)): 
    fitsfilename = os.path.join(datapath2, files[i])    
    hdulist = fits.open(fitsfilename)     
    hdubias = hdulist[0]  
    hdu20 = hdulist[1] 
    hdu40 = hdulist[2] 
    hdu100 = hdulist[3] 
    hdu180 = hdulist[4] 

    fbiasimage[i] = hdubias.data * 1.0
    f20image[i] = hdu20.data * 1.0
    f40image[i] = hdu40.data * 1.0
    f100image[i] = hdu100.data * 1.0
    f180image[i] = hdu180.data * 1.0


    
    
    

In [ ]:
fimage = np.zeros(1024,1024)

for i in range(len(files)): 
    fitsfilename = os.path.join(datapath2, files[i])    
    hdulist = fits.open(fitsfilename)                  
    hdubias = hdulist[0]  
    hdu20 = hdulist[1] 
    hdu40 = hdulist[2] 
    hdu100 = hdulist[3] 
    hdu180 = hdulist[4] 
    
    fimage[i] = hdubias.data * 1.0  + hdu20.data * 2.0 + hdu40.data * 3.0 + hdu100.data * 4.0 + hdu180.data *5.0                 
                      
fpix = np.zeros(1024,1024)
for i in range(len(files))
    fpix += fimage[i]

